<a href="https://colab.research.google.com/github/el-eshaano/ml/blob/main/Transformers/NanoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-02-17 16:25:03--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2024-02-17 16:25:04 (34.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
!pip install icecream

In [3]:
with(open('input.txt', 'r') as f):
    text = f.read()

In [4]:
chars = sorted(list(set(text)))
print(''.join(chars))
print(len(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s : [stoi[c] for c in s]
decode = lambda n : ''.join([itos[d] for d in n])

In [6]:
decode(encode("hello"))

'hello'

In [7]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [8]:
n = int(0.9 * len(data))
train_data = data[:n]
test_data = data[n:]

# Important Concept - `block_size`

The `block_size` specifies the maximum size of the data we will pull for training. It is the maximum number of examples that will be in any subsection of the array that we pull. It represents the maximum context level of our model

So, for example, suppose our array is:
`[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`

Here, if `block_size = 2`, we will pull 3 elements from the array, lets say the first three: `[1, 2, 3]`

This set of 3 elements contains **two** training examples for our transformer,
1. `1 => 2`
2. `1, 2 => 3`

Hence we will always take a subarray of size `block_size + 1`

So we can write
```python
x = test_array[:block_size]
y = test_array[1:block_size+1]
```

Hence, for any given value `t` in the `range(block_size)`
```python
context = x[:t+1]
target = y[t]
```


In [9]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split="train"):
    data = train_data if split == 'train' else test_data

    ix = torch.randint(len(data) - block_size, (batch_size, )) # function call is torch.randint(low=0, high, size, ...)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

xb, yb = get_batch("train")
print(xb.size())
print(xb)
print('---')
print(yb.size())
print(yb)

torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
---
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super(BigramLanguageModel, self).__init__()
        self.embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.embedding_table(idx)

        if targets is None:
            loss = None
        else:

            # In the Embedding layer, the table is of the shape Batch, Time, Channels
            # In our case, B=4, T=8, C=vocab_size=65
            B, T, C = logits.shape

            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            loss = F.cross_entropy(logits, targets)


        return logits, loss

    def generate(self, idx, max_new_tokens):

        # idx is (B, T)
        for _ in range(max_new_tokens):
            logits, loss = self(idx) # (B*T, C)
            # only take the last timestep data
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx

vocab_size = len(chars)
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)

torch.Size([32, 65])


In [19]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


pdcbf?pGXepydZJSrF$Jrqt!:wwWSzPNxbjPiD&Q!a;yNt$Kr$o-gC$WSjJqfBKBySKtSKpwNNfyl&w:q-jluBatD$Lj;?yzyUca


In [20]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [23]:
batch_size = 32
for steps in range(10000):

    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5067548751831055


In [26]:
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


S:
Ad?
Se withind wee ut my had:
ce houeftle quese
F&imat--
Vavou allitouroo-kenged f y aikit,
being'lanered nt.
ORORYee torys, vy, it D:
A wo lllgsthige an, u,
Y incllince bary d t s at m t e, lomy mand itha s ily har,
QYCo otother brt hech J;
Dougeatin t looungoberevy Y avelmithe buch w,

M
Fothou
